In [ ]:
from src.definitions import PROJECT_ROOT
from glob import glob
from pathlib import Path
import json
import spacy
import pandas as pd

pd.options.display.max_columns = 99

In [ ]:
examples = [
    json.loads(
        Path(path).open().read()
    )
    for path in glob(str(
        PROJECT_ROOT / "data/test/*.json"
    ))
]

nlp = spacy.load("en_core_web_sm")

def parse_passage(passage: dict, pubtator_id: str, passage_id: int):
    return pd.DataFrame([
        {
            "pubtator_id": pubtator_id,
            "passage_id": passage_id,
            "words": token.text,
            "labels": [
                annotation["infons"]["type"]
                for annotation in passage["annotations"]
                for location in annotation["locations"]
                if location["offset"] == token.idx
            ],
        }
        for token in nlp(passage["text"])
    ])

df = pd.concat([
    parse_passage(passage, example["id"], idx)
    for example in examples
    for idx, passage in enumerate(example["passages"])
    if passage["text"] != ""
])

processed_dir = PROJECT_ROOT / "data/processed"
processed_dir.mkdir(exist_ok=True)

df.to_csv(
    PROJECT_ROOT / "data/processed/labeled_passages.csv",
    index=False,
)

df.T

In [ ]:
type(nlp)